<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from io import StringIO
import requests
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import re
import json
import squarify
import matplotlib.pyplot as plt
import spacy
from spacy.tokenizer import Tokenizer
from nltk.stem import PorterStemmer

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
nlp = spacy.load("en_core_web_lg")

In [3]:
tokenizer = Tokenizer(nlp.vocab)

In [4]:
yelp.head

<bound method NDFrame.head of                  business_id  cool                date  funny  \
0     nDuEqIyRc8YKS1q1fX0CZg     1 2015-03-31 16:50:30      0   
1     eMYeEapscbKNqUDCx705hg     0 2015-12-16 05:31:03      0   
2     6Q7-wkCPc1KF75jZLOTcMw     1 2010-06-20 19:14:48      1   
3     k3zrItO4l9hwfLRwHBDc9w     3 2010-07-13 00:33:45      4   
4     6hpfRwGlOzbNv7k5eP9rsQ     1 2018-06-30 02:30:01      0   
...                      ...   ...                 ...    ...   
9995  1h3ysSuSazvXc1aeLiiOew     0 2017-10-07 10:57:15      1   
9996  Rwahe1zbFpw6VIjb5ngZeg     0 2014-01-18 15:52:52      0   
9997  8itGZAOBMiTbHKOwLuh4_Q     0 2018-08-26 02:53:21      0   
9998  A5Rkh7UymKm0_Rxm9K2PJw     0 2018-04-23 23:36:07      0   
9999  xtYiHTmunjfCN2sUaQxBjA     0 2016-06-02 23:37:51      0   

                   review_id  stars  \
0     eZs2tpEJtXPwawvHnHZIgQ      1   
1     DoQDWJsNbU0KL1O29l_Xug      4   
2     DDOdGU7zh56yQHmUnL1idQ      3   
3     LfTMUWnfGFMOfOIyJcwLVA     

In [5]:
yelp.shape

(10000, 9)

In [6]:
yelp.head(10)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA
5,Db3CfZWrtG33UZSs8Tdlsg,1,2016-10-23 22:43:56,1,nXYV_0joQEMXYAfNyOPsRw,4,"Tasty, fast casual Latin street food. The men...",1,Gjz2PCbLZ6midk1n_0LaUg
6,gJhMeq2nVH27tz8LqbD3eQ,0,2013-05-20 19:09:43,0,ZA7SRi6fTRWwpo-B9O72qQ,5,This show is absolutely amazing!! What an incr...,0,BeKPVuqX-2at4izqVwUFEg
7,Yt5gK4E9NqVa14WNiQdBlQ,0,2018-07-12 01:19:53,0,4_GnHPkyTirzK6onIKO4jw,4,Came for the Pho and really enjoyed it! We go...,0,PuXpIJzTBQejeBZh9hwynQ
8,c7WsC8SbUcLyZkREzx9dGA,1,2017-09-27 22:10:26,0,XGGHc7pYgOm5s6SWr8NMXA,5,Absolutely the most Unique experience in a nai...,0,NVVknS1I51z8wY5NNrJ6vQ
9,NSifXpsCRvnsBRqrHF9CJA,0,2015-01-25 08:43:15,0,--e66tyhwCE6eoRmcK2w8g,1,Wow. I walked in and sat at the bar for 10 min...,2,J7MsJKJDSA5OGo2-Hn7MbA


In [7]:
yelp.shape

(10000, 9)

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [8]:
yelp['tokens'] = yelp['text'].str\
                         .replace(r'[\-\!\@\#\$\%\^\&\*\(\)\_\+\[\]\;\'\.\,\/\{\}\:\"\<\>\?\|]','')\
                         .str\
                         .replace(r'[^a-zA-Z\s]', '')\
                         .str\
                         .lower()\
                         .str\
                         .split('\s+')\
                         .apply(lambda x: list(set(x)))

In [9]:
yelp.shape

(10000, 10)

In [10]:
yelp.head(10)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id,tokens
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w,"[ca, person, would, be, i, business, time, pho..."
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA,"[parking, sometimes, be, service, good, togo, ..."
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA,"[nevertheless, business, even, checked, foot, ..."
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ,"[off, people, pictures, i, where, they, who, t..."
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA,"[, eating, variety, think, off, would, tofu, i..."
5,Db3CfZWrtG33UZSs8Tdlsg,1,2016-10-23 22:43:56,1,nXYV_0joQEMXYAfNyOPsRw,4,"Tasty, fast casual Latin street food. The men...",1,Gjz2PCbLZ6midk1n_0LaUg,"[people, be, i, tasty, friday, at, good, but, ..."
6,gJhMeq2nVH27tz8LqbD3eQ,0,2013-05-20 19:09:43,0,ZA7SRi6fTRWwpo-B9O72qQ,5,This show is absolutely amazing!! What an incr...,0,BeKPVuqX-2at4izqVwUFEg,"[rode, visuals, talk, show, terrific, absolute..."
7,Yt5gK4E9NqVa14WNiQdBlQ,0,2018-07-12 01:19:53,0,4_GnHPkyTirzK6onIKO4jw,4,Came for the Pho and really enjoyed it! We go...,0,PuXpIJzTBQejeBZh9hwynQ,"[rubber, right, i, meat, is, here, tripe, to, ..."
8,c7WsC8SbUcLyZkREzx9dGA,1,2017-09-27 22:10:26,0,XGGHc7pYgOm5s6SWr8NMXA,5,Absolutely the most Unique experience in a nai...,0,NVVknS1I51z8wY5NNrJ6vQ,"[want, far, splurge, there, absolutely, nail, ..."
9,NSifXpsCRvnsBRqrHF9CJA,0,2015-01-25 08:43:15,0,--e66tyhwCE6eoRmcK2w8g,1,Wow. I walked in and sat at the bar for 10 min...,2,J7MsJKJDSA5OGo2-Hn7MbA,"[bars, think, would, be, i, right, stations, s..."


## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [11]:
yelp['chunk'] = yelp['tokens'].apply(lambda x: " ".join(x)).astype('str')

In [12]:
text = yelp['chunk']

In [13]:
import os
def gather_data():
    """ Produces List of Documents from a Directory
    filefolder (str): a path of .txt files
    returns list of strings
   """
    data = []
    with json.loads('./data/review_sample.json') as f:
        data.append(f.read())
    return data

In [14]:
"""This freezes my notebook - Lost a ton of time with this"""
# data = gather_data()

'This freezes my notebook - Lost a ton of time with this'

In [15]:
vect = CountVectorizer()

In [16]:
vect.fit(text)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [17]:
dtm = vect.transform(text)

In [18]:
dtm.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [19]:
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())
dtm

,aa,aaa,aaaahhhs,aaasssk,aabs,aamco,aand,aaron,aas,ab,...,zuma,zumanity,zumba,zuni,zupas,zuzana,zuzu,zyrtec,zzaplon,zzzzzzzzz
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
tfidf = TfidfVectorizer(stop_words='english')

In [21]:
dtm = tfidf.fit_transform(text)

In [22]:
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [23]:
dist_matrix  = cosine_similarity(dtm)

In [24]:
df = pd.DataFrame(dist_matrix)

In [25]:
# X = X
# Shape must = (X, X)
df.shape

(10000, 10000)

In [26]:
print(df[6924].sort_values(ascending=False)[:2])

6924    1.000000
8844    0.241258
Name: 6924, dtype: float64


In [27]:
nn = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')

In [28]:
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [29]:
nn.kneighbors([dtm.iloc[1002]])

(array([[0.        , 1.        , 1.        , 1.30445569, 1.30992627]]),
 array([[1002, 6204, 6311, 2225,  681]]))

In [30]:
print(text[1000][:6204])

determines i excellent aspects time say perfect with even issues here since to very november concerns adult which frustrating problem all honestly helped takes am care of acne have results has receive products client we plan can treatment great that happy continue treatments a listen seen discussed been she suggested the for and debi my particular


## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [51]:
train, validate, test = np.split(yelp.sample(frac=1), [int(.6*len(yelp)), int(.8*len(yelp))])

In [43]:
vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [44]:
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])

In [56]:
parameters = {
    'vect__max_df': ( 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(train.chunk, train.stars)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    7.1s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:   23.1s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                          

In [58]:
grid_search.best_score_

0.5376666666666666

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [140]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora



ModuleNotFoundError: No module named 'id2word'

Learn the vocubalary of the yelp data:

In [70]:
text = yelp['chunk'].to_string()

In [113]:
token = tokenizer(text)

In [118]:
STOPWORDS
def tokenize(text): 
    return [tokens for tokens in simple_preprocess(text) if tokens not in STOPWORDS]

In [120]:
tokens = tokenize(text)

In [127]:
df_4 = pd.DataFrame(data={'tokens': tokens})

In [128]:
df_4

,tokens
0,ca
1,person
2,business
3,time
4,phone
...,...
48057,cocktails
48058,theft
48059,laughed
48060,disputed


NameError: name 'id2word' is not defined

TypeError: doc2bow expects an array of unicode tokens on input, not a single string

Create a bag of words representation of the entire corpus

In [ ]:
corpus = ...

Your LDA model should be ready for estimation: 

In [ ]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)